## PycudaDecon deconvolution

Images require padding before passing them to pycudadecon. Otherwise, the shape of deconvolved image will be different than input image. We use a padding where the final dimensions are a multiple of 64. 

cropping psf deteriorates decon quality with pycudadecon
seems ok with RLFish

In [7]:
import numpy as np
import math 
from pycudadecon import RLContext,TemporaryOTF,rl_decon
from aicsimageio import AICSImage
from skimage.io import imread

#Arbitrary image and psf shapes to ensure images for now
#can try different shapes to verify shape of output image

psf = imread("../sample_data/psfs/zeiss_simulated/488.tif")
print(psf.shape)

#from napari_lattice.utils import crop_psf
#psf_1 = crop_psf(psf_img,3e-3)

image = AICSImage("../sample_data/RBC_tiny.czi")
image = np.squeeze(image.data)
print(image.shape)
dzdata=0.3
dxdata=0.1449922
dzpsf=0.3
dxpsf=0.1449922


(93, 205, 205)
(834, 118, 209)


Without padding, if we use pycuda_decon, the deconvolved images get cropped.

In [15]:

with TemporaryOTF(psf) as otf:
    with RLContext(rawdata_shape=image.shape, otfpath=otf.path, dzdata=dzdata, dxdata=dxdata,dzpsf=dzpsf,dxpsf=dxpsf) as ctx:
        decon_res_no_crop = rl_decon(im=image, output_shape = ctx.out_shape,n_iters=10)

print(f"Image shape is {image.shape}. Decon image shape is {decon_res_no_crop.shape}")


Image shape is (834, 118, 209). Decon image shape is (810, 112, 200)


Crop PSF

In [22]:
from napari_lattice.utils import crop_psf

psf_norm = psf/psf.max()
threshold = 1e-4
psf_crop_norm= crop_psf(psf_norm,threshold)
print(psf_crop_norm.shape)

(90, 201, 64)


In [23]:
psf_crop= crop_psf(psf,1e-3)
psf_crop.shape

(58, 112, 34)

In [18]:

with TemporaryOTF(psf_crop) as otf:
    with RLContext(rawdata_shape=image.shape, otfpath=otf.path, dzdata=dzdata, dxdata=dxdata,dzpsf=dzpsf,dxpsf=dxpsf) as ctx:
        decon_res = rl_decon(im=image, output_shape = ctx.out_shape,n_iters=10)

print(f"Image shape is {image.shape}. Decon image shape is {decon_res.shape}")

Image shape is (834, 118, 209). Decon image shape is (810, 112, 200)


In [21]:
with TemporaryOTF(psf_crop_norm) as otf:
    with RLContext(rawdata_shape=image.shape, otfpath=otf.path, dzdata=dzdata, dxdata=dxdata,dzpsf=dzpsf,dxpsf=dxpsf) as ctx:
        decon_res_crop_norm = rl_decon(im=image, output_shape = ctx.out_shape,n_iters=10)

print(f"Image shape is {image.shape}. Decon image shape is {decon_res_crop_norm.shape}")

Image shape is (834, 118, 209). Decon image shape is (810, 112, 200)


In [25]:
#no padding
import RedLionfishDeconv as rl

img_decon_large_crop= rl.doRLDeconvolutionFromNpArrays(data_np = image, 
                                                    psf_np = psf_crop, 
                                                    niter= 10, 
                                                    method = "gpu")

In [26]:
import napari 
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_image(decon_res)
viewer.add_image(decon_res_no_crop)
viewer.add_image(img_decon_large_crop)

c:\Users\Pradeep\.conda\envs\napari_lattice\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'img_decon_large_crop' at 0x240382e3fa0>

In [6]:
def pad_image_nearest_multiple(img:np.ndarray,nearest_multiple:int):
    """pad an Image to the nearest multiple of provided number

    Args:
        img (np.ndarray): 
        nearest_multiple (int): Multiple of number to be padded

    Returns:
        np.ndarray: Padded image
    """    
    rounded_shape = tuple([math.ceil(dim/nearest_multiple)*nearest_multiple for dim in img.shape])
    #get required padding
    padding = np.array(rounded_shape) - np.array(img.shape)
    padded_img = np.pad(img,((0,padding[0]),(0,padding[1]),(0,padding[2])),mode="reflect")
    return padded_img


#pad PSF by dimensions that are multiple of 16
psf_pad = pad_image_nearest_multiple(psf,16)

print(f"PSF shape is {psf.shape}. Padded PSF is {psf_pad.shape}")

#pad image y dimensions that are multiple of 64 and also pad by half of shape
z_psf_pad,y_psf_pad,x_psf_pad = np.array(psf_pad.shape) //2

orig_shape = image.shape
image = np.pad(image,((z_psf_pad,z_psf_pad),(y_psf_pad,y_psf_pad),(x_psf_pad,x_psf_pad)),mode="reflect")

image = pad_image_nearest_multiple(image,64)

print(f"Image shape is {orig_shape}. Padded image is {image.shape}")

PSF shape is (30, 105, 105). Padded PSF is (32, 112, 112)
(320, 448, 768)
(352, 560, 880)
(384, 576, 896)
Image shape is (320, 448, 768). Padded image is (384, 576, 896)


In [7]:
z_psf_pad,y_psf_pad,x_psf_pad

(16, 56, 56)

In [4]:
import napari 
viewer = napari.Viewer()

viewer.add_image(image)
viewer.add_image(orig_image)

c:\Users\deepu\Anaconda3\envs\napari_lattice\lib\site-packages\napari_tools_menu\__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'orig_image' at 0x2c4568eed90>

Run pycudadecon on images with padding

In [3]:
with TemporaryOTF(psf_pad) as otf:
    with RLContext(rawdata_shape=image.shape, otfpath=otf.path, dzdata=dzdata, dxdata=dxdata,dzpsf=dzpsf,dxpsf=dxpsf) as ctx:
        decon_res = rl_decon(im=image, output_shape = ctx.out_shape,n_iters=10)

print(f"Image shape is {image.shape}. Decon image shape is {decon_res.shape}")

Image shape is (256, 320, 640). Decon image shape is (256, 320, 640)


What if we use it on original psf without padding?

In [5]:
with TemporaryOTF(psf) as otf:
    with RLContext(rawdata_shape=img_pad.shape, otfpath=otf.path, dzdata=dzdata, dxdata=dxdata,dzpsf=dzpsf,dxpsf=dxpsf) as ctx:
        decon_res = rl_decon(im=img_pad, output_shape = ctx.out_shape,n_iters=10)

print(f"Image shape is {img_pad.shape}. Decon image shape is {decon_res.shape}")

c:\Users\deepu\Anaconda3\envs\napari_lattice\lib\site-packages\pycudadecon\otf.py:59: RuntimeWarning: divide by zero encountered in longlong_scalars
  outnx = max_otf_size // (outnz * 4) - 2


Image shape is (1152, 128, 256). Decon image shape is (1152, 128, 256)


It seems to work, however, we'll use psf with some padding as we've noticed some discrepancies occasionally